# Proyecto 2
## RSNA 2022 Cervical Spine Fracture Detection
**Universidad del Valle de Guatemala**\
**Facultad de Ingeniería**\
**Departamento de Ciencias de la Computación**\
**Data Science**

# Generador de Metadatos
---

## Integrantes
- Gustavo Gonzalez
- Pablo Orellana
- Diego Leiva
- Maria Ramirez

---


## Librerias

In [1]:
# Operaciones de archivos
import os
from glob import glob

# Operaciones de datos
import pandas as pd
import numpy as np

# Operaciones de imágenes
import nibabel as nib

# Visualización
from tqdm import tqdm

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Metadatos DICOM

En la fase de análisis exploratorio se descubrió que las imágenes DICOM contienen etiquetas relevantes asociadas a metadatos específicos de la tomografía. Por lo tanto, se extraerá la información relevante para utilizar estos metadatos en una posterior detección de vértebras, aprovechando la información proporcionada.

## Lectura de Metadatos

In [2]:
meta_train = pd.read_csv("data/meta_train.csv")
print("Metadata cargado exitosamente")

Metadata cargado exitosamente


In [3]:
meta_train.head()

,Rows,Columns,SOPInstanceUID,ContentDate,SliceThickness,InstanceNumber,ImagePositionPatient,ImageOrientationPatient
0,512,512,1.2.826.0.1.3680043.6200.1.1,20220727,1.0,1,"[-118.530, -103.5302, 334.50]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."
1,512,512,1.2.826.0.1.3680043.6200.1.10,20220727,1.0,10,"[-118.530, -103.5302, 327.30]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."
2,512,512,1.2.826.0.1.3680043.6200.1.100,20220727,1.0,100,"[-118.530, -103.5302, 255.30]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."
3,512,512,1.2.826.0.1.3680043.6200.1.101,20220727,1.0,101,"[-118.530, -103.5302, 254.50]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."
4,512,512,1.2.826.0.1.3680043.6200.1.102,20220727,1.0,102,"[-118.530, -103.5302, 253.70]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."


## Procesamiento de Metadatos

### Columnas de Identificación y Tamaño de Imagen

In [4]:
if not os.path.exists("metadata/dicom_metadata.csv"):
    # Generar columna del ID del estudio a partir del ID de la imagen
    meta_train["StudyInstanceUID"] = meta_train["SOPInstanceUID"].apply(lambda x: ".".join(x.split(".")[:-2]))
    print("ID de estudio generado exitosamente")

    # Generar columna del tamaño de la imagen a partir de las filas y columnas
    meta_train["ImageSize"] = meta_train["Rows"].astype(str) + " x " + meta_train["Columns"].astype(str)
    print("Tamaño de imagen generado exitosamente")
else:
    meta_train_clean = pd.read_csv("metadata/dicom_metadata.csv")
    print("Metadata de DICOM ya existe, cargando...")

ID de estudio generado exitosamente
Tamaño de imagen generado exitosamente


### Extracción de Coordenadas de Posición del Paciente

In [5]:
if not os.path.exists("metadata/dicom_metadata.csv"):
    # Extraer las coordenadas de la posición de la imagen en los ejes x, y, z
    meta_train['ImagePositionPatient_x'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[0]))
    meta_train['ImagePositionPatient_y'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[1]))
    meta_train['ImagePositionPatient_z'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[2]))
    print("Coordenadas de la posición de la imagen extraídas exitosamente")
else:
    print("Metadata de DICOM ya existe, omitiendo...")

Coordenadas de la posición de la imagen extraídas exitosamente


In [6]:
if not os.path.exists("metadata/dicom_metadata.csv"):
    # Eliminar columnas innecesarias reemplazadas por las nuevas
    meta_train_clean = meta_train.drop(['SOPInstanceUID','ImagePositionPatient','ImageOrientationPatient','ImageSize','ContentDate'], axis=1)
    # Renombrar columnas para mayor claridad
    meta_train_clean.rename(columns={"Rows": "ImageHeight", "Columns": "ImageWidth","InstanceNumber": "Slice"}, inplace=True)
    # Reordenar las columnas
    meta_train_clean = meta_train_clean[['StudyInstanceUID','Slice','ImageHeight','ImageWidth','SliceThickness','ImagePositionPatient_x','ImagePositionPatient_y','ImagePositionPatient_z']]
    # Ordenar los datos por ID de estudio y número de corte
    meta_train_clean.sort_values(by=['StudyInstanceUID','Slice'], inplace=True)
    # Resetear el índice
    meta_train_clean.reset_index(drop=True, inplace=True)
    print("Metadata limpiado exitosamente")
else:
    print("Metadata de DICOM ya existe, omitiendo...")

Metadata limpiado exitosamente


In [7]:
meta_train_clean.head()

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z
0,1.2.826.0.1.3680043.10001,1,512,512,0.625,-52.308,-27.712,7.282
1,1.2.826.0.1.3680043.10001,2,512,512,0.625,-52.308,-27.712,6.657
2,1.2.826.0.1.3680043.10001,3,512,512,0.625,-52.308,-27.712,6.032
3,1.2.826.0.1.3680043.10001,4,512,512,0.625,-52.308,-27.712,5.407
4,1.2.826.0.1.3680043.10001,5,512,512,0.625,-52.308,-27.712,4.782


## Exportacion de Metadatos Limpios

In [8]:
if not os.path.exists("metadata/dicom_metadata.csv"):
    # Guardar el archivo limpio en formato CSV
    meta_train_clean.to_csv("metadata/dicom_metadata.csv", index=False)
    print("Metadata guardado exitosamente")
else:
    print("Metadata de DICOM ya existe, omitiendo...")

Metadata guardado exitosamente


# Metadatos NIFTI

En la fase de análisis se descubrió que los archivos de segmentación contienen metadatos asociados que permiten identificar las vértebras visualizadas en las tomografías DICOM. Por lo tanto, extraer y almacenar estos metadatos es fundamental para poder inferir las vértebras visibles en cada corte, especialmente en aquellos pacientes que no cuentan con segmentaciones.

## Obtencion de Segmentaciones

In [9]:
# Definir ruta base y obtener rutas de segmentaciones
base_path = "data"
seg_paths = glob(f"{base_path}/segmentations/*")

# Crear DataFrame y extraer el UID limpio
seg_df = pd.DataFrame({'path': seg_paths})
seg_df['StudyInstanceUID'] = seg_df['path'].apply(lambda x: os.path.basename(x).replace('.nii', ''))

# Estandarizar el separador de ruta a "/"
seg_df['path'] = seg_df['path'].apply(lambda x: x.replace("\\", "/"))

# Reordenar columnas y mostrar resultados
seg_df = seg_df[['StudyInstanceUID', 'path']]
print(f"Existen un total de {seg_df.shape[0]} pacientes con segmentaciones")
seg_df.head()

Existen un total de 87 pacientes con segmentaciones


,StudyInstanceUID,path
0,1.2.826.0.1.3680043.10633,data/segmentations/1.2.826.0.1.3680043.10633.nii
1,1.2.826.0.1.3680043.10921,data/segmentations/1.2.826.0.1.3680043.10921.nii
2,1.2.826.0.1.3680043.11827,data/segmentations/1.2.826.0.1.3680043.11827.nii
3,1.2.826.0.1.3680043.11988,data/segmentations/1.2.826.0.1.3680043.11988.nii
4,1.2.826.0.1.3680043.12281,data/segmentations/1.2.826.0.1.3680043.12281.nii


## Seleccion de Pacientes con Segmentaciones

In [10]:
# Cargar el archivo de metadatos
meta_train = pd.read_csv("metadata/dicom_metadata.csv")

# Filtrar los metadatos de entrenamiento 
# para obtener solo los estudios con segmentaciones
meta_seg = meta_train[meta_train['StudyInstanceUID'].isin(seg_df['StudyInstanceUID'])].reset_index(drop=True)
print(f"Los metadatos consisten de {meta_seg.shape[1]} columnas")
print(f"Hay un total de {meta_seg.shape[0]} instancias de segmentación")
meta_seg.head()

Los metadatos consisten de 8 columnas
Hay un total de 29832 instancias de segmentación


,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z
0,1.2.826.0.1.3680043.10633,1,512,512,1.0,-68.0,98.0,314.099976
1,1.2.826.0.1.3680043.10633,2,512,512,1.0,-68.0,98.0,313.599976
2,1.2.826.0.1.3680043.10633,3,512,512,1.0,-68.0,98.0,313.099976
3,1.2.826.0.1.3680043.10633,4,512,512,1.0,-68.0,98.0,312.599976
4,1.2.826.0.1.3680043.10633,5,512,512,1.0,-68.0,98.0,312.099976


## Extraccion de vertebras 

In [11]:
# Inicializar las columnas de segmentación en 0
targets = ['C1','C2','C3','C4','C5','C6','C7']
meta_seg[targets]=0
k=0

In [12]:
# Iterar sobre cada archivo de segmentación con tqdm
for path, UID in tqdm(zip(seg_df['path'], seg_df['StudyInstanceUID']), total=len(seg_df), desc="Procesando segmentaciones", unit="paciente"):
    # Obtener segmentación del paciente
    seg_nib = nib.load(path)
    seg = seg_nib.get_fdata()

    # Ajustar orientación
    seg = seg[:, ::-1, ::-1].transpose(2, 1, 0)
    num_slices, _, _ = seg.shape
    
    # Iterar sobre los cortes con barra de progreso interna
    for i in tqdm(range(num_slices), leave=False, desc=f"Segmentación {UID}"):
        mask = seg[i]
        unique_vals = np.unique(mask)
        
        # Iterar sobre los valores únicos (excepto 0)
        for j in unique_vals[1:]:
            # Ignorar valores mayores a 7 (Vértebras torácicas, lumbares y sacras)
            if j <= 7:
                meta_seg.loc[(meta_seg['StudyInstanceUID'] == UID) & (meta_seg['Slice'] == i), f'C{int(j)}'] = 1

print("Segmentaciones procesadas exitosamente")

Procesando segmentaciones: 100%|██████████| 87/87 [08:36<00:00,  5.94s/paciente]

Segmentaciones procesadas exitosamente


In [13]:
meta_seg.head()

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.10633,1,512,512,1.0,-68.0,98.0,314.099976,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.10633,2,512,512,1.0,-68.0,98.0,313.599976,0,0,0,0,0,0,0
2,1.2.826.0.1.3680043.10633,3,512,512,1.0,-68.0,98.0,313.099976,0,0,0,0,0,0,0
3,1.2.826.0.1.3680043.10633,4,512,512,1.0,-68.0,98.0,312.599976,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.10633,5,512,512,1.0,-68.0,98.0,312.099976,0,0,0,0,0,0,0


## Exportacion de Metadatos Limpios

In [14]:
meta_seg.to_csv("metadata/segmentation_metadata.csv", index=False)
print("Metadata de segmentación guardado exitosamente")

Metadata de segmentación guardado exitosamente


In [15]:
# Ejemplo de segmentación
meta_seg[['StudyInstanceUID','Slice']+targets].iloc[199:204,:]

,StudyInstanceUID,Slice,C1,C2,C3,C4,C5,C6,C7
199,1.2.826.0.1.3680043.10633,200,0,1,1,0,0,0,0
200,1.2.826.0.1.3680043.10633,201,0,1,1,0,0,0,0
201,1.2.826.0.1.3680043.10633,202,0,0,1,1,0,0,0
202,1.2.826.0.1.3680043.10633,203,0,0,1,1,0,0,0
203,1.2.826.0.1.3680043.10633,204,0,0,1,1,0,0,0
